# Интерфейсы scikit-learn

In [85]:
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [86]:
import yaml

with open('../config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

## Estimator

Для примера построим простой estimator, который в перспективе будет вычитать из признаков их среднее значение и после сдвигать признаки на заранее заданную константу

In [87]:
class SubtractMeanAndShiftEstimator(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

In [88]:
m = SubtractMeanAndShiftEstimator(shift=3)

Метод `get_params()` реализован в `BaseEstimator`, и мы можем сразу использовать его для получения гиперпараметров модели. Это возможно, так как единственный гиперпараметр `shift` был передан как явное ключевое слово в контрукторе

Обратите внимание, что соответствующий аттрибут класса должен совпадать с ключевым словом: `self.shift = shift`

In [89]:
m.get_params()

{'shift': 3}

Аналогично мы можем использовать `set_params()` для задания значений гиперпараметров. Этот метод пригодится при поиске оптимальных значений гиперпараметров

In [90]:
m.set_params(shift=5)
m.get_params()

{'shift': 5}

In [91]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [ 0, -8],
    [ 2, 10],
    [ 1,  1],
])
m.fit(X, y)

SubtractMeanAndShiftEstimator(shift=5)

В sklearn есть класс sklearn.base.OutlierMixin, который позволяет реализовывать кастомные классы для определения выбросов.
Он добавляет:
- атрибут _estimator_type, по умолчанию outlier_detector
- fit_predict.

Метод fit() работает в формате без учителя, predict же должен классифицировать данные на аутлаеры (возвращать для них -1) и обыычные данные (возвращать 1). Для классификации используется отсечка по порогу предсказаний, полученных внутренним.
Во встроенных методах функция оценки доступна с помощью метода `score_samples`, в то время как порог можно задать параметром `contamination`. 
Например, для гауссовских данных можно использовать sklearn.covariance.EllipticEnvelope.

**Задание**: Создайте свой эстиматор с использованием sklearn.base.OutlierMixin, который будет определять выбросы на основе интерквартильного размаха. 
Он должен возвращать один столбец с 1 и -1, а также позволять задавать порог для квантиля, определяющего размах. Не забудьте, что он должен быть двухсторонним.
Ваш эстиматор должен работать и для датафреймов, и для numpy массивов.

In [92]:
from sklearn.base import BaseEstimator, OutlierMixin
class MyEstimator(BaseEstimator, OutlierMixin):
    def __init__(self, factor=1.5, percent_q1=0.25, percent_q3=0.75):
        self.factor = factor
        self.percent_q1 = percent_q1
        self.percent_q3 = percent_q3

    def fit(self, X, y=None):
         self.is_fitted_ = True
         X = pd.DataFrame(X.copy())
         q1 = X.quantile(self.percent_q1)
         q3 = X.quantile(self.percent_q3)
         iqr = q3 - q1
         self._lower_bound = q1 - (self.factor * iqr)
         self._upper_bound = q3 + (self.factor * iqr)
         print(X.min(axis=0))
         return self
         
    def predict(self, X):
        X = pd.DataFrame(X.copy())
        temp = np.ones(len(X))
        temp[(X > self._lower_bound.values).all(axis=1) & (X < self._upper_bound.values).all(axis=1)] = -1
        return temp

In [93]:
estimator = MyEstimator()
true_cov = np.array([[.8, .3],
                     [.3, .4]])
X = np.random.RandomState(42).multivariate_normal(mean=[0, 0],
                                                 cov=true_cov,
                                                 size=500)
e = estimator.fit(X)
pred = estimator.predict(X)
e._lower_bound

0   -2.918998
1   -1.876028
dtype: float64


0   -2.280408
1   -1.667910
dtype: float64

In [94]:
e._upper_bound

0    2.300159
1    1.708542
dtype: float64

In [95]:
e.predict(X)

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

## Predictor

Рассмотрим тот же класс, но добавим к нему методы `predict()` и `score()`

In [96]:
class SubtractMeanAndShiftPredictor(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def predict(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

    def score(self, X: NDArray, y: NDArray) -> float:
        return r2_score(y, self.predict(X))  # R2 \in (-\infty; 1] is the coefficient of determination

Так как мы специально добавили небольшое отклонение в y, наш R2 чуть меньше 1

In [101]:
model = SubtractMeanAndShiftPredictor(shift=1)
model.fit(X)
model.predict(X)

array([[ 6.13916070e-01,  6.97177225e-01],
       [ 9.98733908e-02,  1.34409129e+00],
       [ 1.26760315e+00,  9.93810430e-01],
       [-5.30603943e-01,  5.87313353e-01],
       [ 1.29161135e+00,  1.43772751e+00],
       [ 1.51918650e+00,  9.99919319e-01],
       [ 1.24389177e+00,  4.91928332e-02],
       [ 2.63155718e+00,  1.54165230e+00],
       [ 1.81384802e+00,  1.59050612e+00],
       [ 2.12193465e+00,  7.97150035e-01],
       [-2.03144819e-01,  2.11330185e-01],
       [ 1.28182267e+00,  3.40629036e-01],
       [ 1.45600327e+00,  1.28614781e+00],
       [ 1.91904706e+00,  1.68091218e+00],
       [ 1.59757666e+00,  1.13846277e+00],
       [ 1.10346764e+00,  2.06393206e+00],
       [ 1.26709066e+00,  5.36451064e-01],
       [ 5.82324033e-01,  7.94361235e-02],
       [ 1.28320388e+00,  4.44855641e-02],
       [ 2.11345348e+00,  1.68570059e+00],
       [ 3.33522634e-01,  7.18961873e-01],
       [ 1.18066387e+00,  9.10115278e-01],
       [ 2.45502286e+00,  1.35972974e+00],
       [ 1.

## Transformer

Рассмотрим тот же класс, но добавим к нему метод `transform()`

In [132]:
class SubtractMeanAndShiftTransformer(BaseEstimator, OneToOneFeatureMixin, TransformerMixin):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()
        self.n_features_in_ = None

    def fit(self, X: NDArray, y: NDArray = None):
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        self.n_features_in_ = X.shape[1]  # Сохраняем количество признаков
        return self

    def transform(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

In [133]:
t = SubtractMeanAndShiftTransformer(shift=5)
t.fit(X)
t.transform(X)

array([[4.61391607, 4.69717723],
       [4.09987339, 5.34409129],
       [5.26760315, 4.99381043],
       [3.46939606, 4.58731335],
       [5.29161135, 5.43772751],
       [5.5191865 , 4.99991932],
       [5.24389177, 4.04919283],
       [6.63155718, 5.5416523 ],
       [5.81384802, 5.59050612],
       [6.12193465, 4.79715004],
       [3.79685518, 4.21133018],
       [5.28182267, 4.34062904],
       [5.45600327, 5.28614781],
       [5.91904706, 5.68091218],
       [5.59757666, 5.13846277],
       [5.10346764, 6.06393206],
       [5.26709066, 4.53645106],
       [4.58232403, 4.07943612],
       [5.28320388, 4.04448556],
       [6.11345348, 5.68570059],
       [4.33352263, 4.71896187],
       [5.18066387, 4.91011528],
       [6.45502286, 5.35972974],
       [5.1651667 , 5.65563936],
       [5.12136006, 4.06699938],
       [4.82007666, 4.67052659],
       [5.45491051, 5.56348222],
       [3.90528209, 4.91152698],
       [5.80778018, 5.24123741],
       [4.49971111, 5.25422289],
       [5.

Так как мы добавили `TransformerMixin`, мы можем использовать метод `fit_transform()`, не реализуя его явно

In [134]:
t.fit_transform(X)

array([[4.61391607, 4.69717723],
       [4.09987339, 5.34409129],
       [5.26760315, 4.99381043],
       [3.46939606, 4.58731335],
       [5.29161135, 5.43772751],
       [5.5191865 , 4.99991932],
       [5.24389177, 4.04919283],
       [6.63155718, 5.5416523 ],
       [5.81384802, 5.59050612],
       [6.12193465, 4.79715004],
       [3.79685518, 4.21133018],
       [5.28182267, 4.34062904],
       [5.45600327, 5.28614781],
       [5.91904706, 5.68091218],
       [5.59757666, 5.13846277],
       [5.10346764, 6.06393206],
       [5.26709066, 4.53645106],
       [4.58232403, 4.07943612],
       [5.28320388, 4.04448556],
       [6.11345348, 5.68570059],
       [4.33352263, 4.71896187],
       [5.18066387, 4.91011528],
       [6.45502286, 5.35972974],
       [5.1651667 , 5.65563936],
       [5.12136006, 4.06699938],
       [4.82007666, 4.67052659],
       [5.45491051, 5.56348222],
       [3.90528209, 4.91152698],
       [5.80778018, 5.24123741],
       [4.49971111, 5.25422289],
       [5.

Аналогично мы можем использовать метод `get_feature_names_out()`, так как мы добавили `OneToOneFeatureMixin`

In [135]:
t.get_feature_names_out(input_features=['x', 'y'])

array(['x', 'y'], dtype=object)

Используем датасет с домами как пример. Вспомним, что мы делали в прошлый раз, и попробуем заполнить пропущенные значения в некоторых числовызх столбцах.
Для этого используем трансформер по столбцам. 

In [136]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


По умолчанию, только указанные столбцы трансформируются и возвращаются (remainder=`drop`). Мы же сделаем так, чтобы все остальные столбцы тоже возвращались, просто с ними бы ничего не делалось. 

In [137]:
ct = ColumnTransformer(
    [('mean_impute', SimpleImputer(strategy='mean'), ['SalePrice', 'LotArea', 'WoodDeckSF',  'MasVnrArea'])], 
    remainder="passthrough")

ct.fit(df)

C:\Users\glugo\AppData\Roaming\Python\Python313\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('mean_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea'])])

In [138]:
ct.transform(df)

array([[208500.0, 8450.0, 0.0, ..., 2008, 'WD', 'Normal'],
       [181500.0, 9600.0, 298.0, ..., 2007, 'WD', 'Normal'],
       [223500.0, 11250.0, 0.0, ..., 2008, 'WD', 'Normal'],
       ...,
       [266500.0, 9042.0, 0.0, ..., 2010, 'WD', 'Normal'],
       [142125.0, 9717.0, 366.0, ..., 2010, 'WD', 'Normal'],
       [147500.0, 9937.0, 736.0, ..., 2008, 'WD', 'Normal']], dtype=object)

In [139]:
ct.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('mean_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea'])])

Если у датасета появятсся столбцы, которые не были представлены во время fit (даже среди тех, что не трансформировались), то они будут выкинуты на этапе transform. 

In [140]:
df["temp"] = 0
ct.transform(df)

,mean_impute__SalePrice,mean_impute__LotArea,mean_impute__WoodDeckSF,mean_impute__MasVnrArea,remainder__Id,remainder__MSSubClass,remainder__MSZoning,remainder__LotFrontage,remainder__Street,remainder__Alley,...,remainder__ScreenPorch,remainder__PoolArea,remainder__PoolQC,remainder__Fence,remainder__MiscFeature,remainder__MiscVal,remainder__MoSold,remainder__YrSold,remainder__SaleType,remainder__SaleCondition
0,208500.0,8450.0,0.0,196.0,1,60,RL,65.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,181500.0,9600.0,298.0,0.0,2,20,RL,80.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,223500.0,11250.0,0.0,162.0,3,60,RL,68.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,140000.0,9550.0,0.0,0.0,4,70,RL,60.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,250000.0,14260.0,192.0,350.0,5,60,RL,84.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,175000.0,7917.0,0.0,0.0,1456,60,RL,62.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,210000.0,13175.0,349.0,119.0,1457,20,RL,85.0,Pave,NaN,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,266500.0,9042.0,0.0,0.0,1458,70,RL,66.0,Pave,NaN,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,142125.0,9717.0,366.0,0.0,1459,20,RL,68.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


**Задание**: Перейдите к медианному заполнению пропусков. Проверьте, что результаты, полученные с помощью трансформации, соответствуют преобразованию напрямую.

In [162]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])

columns = ['SalePrice', 'LotArea', 'WoodDeckSF',  'MasVnrArea']

ct_median = ColumnTransformer(
    [('median_impute', SimpleImputer(strategy='median'), columns)], 
    remainder="passthrough", verbose_feature_names_out=False)

ct_median.set_output(transform='pandas')
df_transformed = ct_median.fit_transform(df)

df_copy = df.copy()
df_copy[columns] = df_copy[columns].fillna(df_copy[columns].median())

df_copy = df_copy[df_transformed.columns]

diff = df_transformed.compare(df_copy)
print(diff)

Empty DataFrame
Columns: []
Index: []


**Задание**: Добавьте еще нормализатор для LotFrontage, LotArea и запустите в ColumnTransformer. Обучите его и примените.

In [178]:
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(cfg['house_pricing']['train_dataset'])

columns_impute = ['SalePrice', 'LotArea', 'WoodDeckSF', 'MasVnrArea']
columns_scale = ['LotFrontage', 'LotArea']

ct_median = ColumnTransformer(
    [
        ('median_impute', SimpleImputer(strategy='median'), columns_impute),
        ('normalize', StandardScaler(), columns_scale)
    ], 
    remainder="passthrough", verbose_feature_names_out=True)

ct_median.set_output(transform='pandas')
df_transformed = ct_median.fit_transform(df)

df_copy = df.copy()
df_copy[[f'median_impute__{col}' for col in columns_impute]] = df_copy[columns_impute].fillna(df_copy[columns_impute].median())

scaler = StandardScaler()
df_copy[[f'normalize__{col}' for col in columns_scale]] = scaler.fit_transform(df_copy[columns_scale])

changed_columns = columns_impute + columns_scale

remaining_columns = [col for col in df.columns if col not in changed_columns]
df_copy[[f'remainder__{col}' for col in remaining_columns]] = df_copy[remaining_columns]

df_copy = df_copy[[f"median_impute__{col}" for col in columns_impute] + 
                  [f"normalize__{col}" for col in columns_scale] +
                  [f"remainder__{col}" for col in remaining_columns]]

df_transformed.equals(df_copy)

False

Sklearn располагает большим количеством встроенных трансформеров. Соответствующие трансформеры есть и для категориальных фичей (более подробно рассмотрим этот тип чуть позже). Например, известное нам бинарное кодирование можно проводить с помощью OneHotEncoder()

In [179]:
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(
    transformers=[
        ('median_impute', SimpleImputer(strategy='mean'), ['SalePrice', 'LotArea', 'WoodDeckSF',  'MasVnrArea']),
        ("one_hot_encode", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["MSZoning", "SaleType", "SaleCondition"]),
    ], 
    remainder="passthrough")

ct.fit(df)

C:\Users\glugo\AppData\Roaming\Python\Python313\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('median_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea']),
                                ('one_hot_encode',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['MSZoning', 'SaleType', 'SaleCondition'])])

In [180]:
ct.set_output(transform='pandas')
ct.transform(df)

,median_impute__SalePrice,median_impute__LotArea,median_impute__WoodDeckSF,median_impute__MasVnrArea,one_hot_encode__MSZoning_C (all),one_hot_encode__MSZoning_FV,one_hot_encode__MSZoning_RH,one_hot_encode__MSZoning_RL,one_hot_encode__MSZoning_RM,one_hot_encode__SaleType_COD,...,remainder__EnclosedPorch,remainder__3SsnPorch,remainder__ScreenPorch,remainder__PoolArea,remainder__PoolQC,remainder__Fence,remainder__MiscFeature,remainder__MiscVal,remainder__MoSold,remainder__YrSold
0,208500.0,8450.0,0.0,196.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,0,2,2008
1,181500.0,9600.0,298.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,0,5,2007
2,223500.0,11250.0,0.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,0,9,2008
3,140000.0,9550.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,272,0,0,0,NaN,NaN,NaN,0,2,2006
4,250000.0,14260.0,192.0,350.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,175000.0,7917.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,0,8,2007
1456,210000.0,13175.0,349.0,119.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,MnPrv,NaN,0,2,2010
1457,266500.0,9042.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010
1458,142125.0,9717.0,366.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,112,0,0,0,NaN,NaN,NaN,0,4,2010


Для выбора столбцов можно создавать make_selector, например, по выбору численных и категориальных значений. 

**Доп. задание**. Сделайте трансформер для OneHotEncoder на основе make_selector так, чтобы выбирать все нечисловые столбцы. Сколько столбцов получается после трансформации?

In [ ]:
from sklearn.compose import make_column_selector as selector

# Your code

## Pipelines

С помощью Pipeline мы можем производить последовательную обработку данных и выполнять предсказание в конце

In [182]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [0],
    [2],
    [1],
])

pipeline = Pipeline([
    ("shifter", SubtractMeanAndShiftTransformer(shift=5)),
    ("regressor", LinearRegression()),
])
...
pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)

[[-4.4408921e-16]
 [ 2.0000000e+00]
 [ 1.0000000e+00]]


Pipeline хранит последовательные Estimators в аттрибуте `steps`

In [183]:
pipeline.steps

[('shifter', SubtractMeanAndShiftTransformer(shift=5)),
 ('regressor', LinearRegression())]

Перейти к объекту i-го Estimator можно напрямую через `pipeline[i]`:

In [184]:
pipeline[0]

SubtractMeanAndShiftTransformer(shift=5)

In [185]:
pipeline[1].coef_

array([[0.00990099, 0.0990099 ]])

Так как Pipeline сам является Estimator, мы можем увидеть список его параметров:

In [186]:
pipeline.get_params()

{'memory': None,
 'steps': [('shifter', SubtractMeanAndShiftTransformer(shift=5)),
  ('regressor', LinearRegression())],
 'transform_input': None,
 'verbose': False,
 'shifter': SubtractMeanAndShiftTransformer(shift=5),
 'regressor': LinearRegression(),
 'shifter__shift': 5,
 'regressor__copy_X': True,
 'regressor__fit_intercept': True,
 'regressor__n_jobs': None,
 'regressor__positive': False}

Видно, параметры промежуточных Estimator указаны как `<estimator>__<parameter>`. Следовательно, мы можем изменить параметры любого промежуточного Estimator:

In [187]:
pipeline.set_params(shifter__shift=10)
pipeline.get_params()

{'memory': None,
 'steps': [('shifter', SubtractMeanAndShiftTransformer(shift=10)),
  ('regressor', LinearRegression())],
 'transform_input': None,
 'verbose': False,
 'shifter': SubtractMeanAndShiftTransformer(shift=10),
 'regressor': LinearRegression(),
 'shifter__shift': 10,
 'regressor__copy_X': True,
 'regressor__fit_intercept': True,
 'regressor__n_jobs': None,
 'regressor__positive': False}

**Задание**: Создайте пайплайн по преобразованию чсиленных столбцов, содержащий импьютер и скейлер.

In [202]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])

numeric_features = df.select_dtypes(include=['number']).columns

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', cat_pipeline, numeric_features)
], remainder='passthrough')

preprocessor.set_output(transform='pandas')
df_transformed = preprocessor.fit_transform(df)

# print(df)
print(df_transformed)

       num__Id  num__MSSubClass  num__LotFrontage  num__LotArea  \
0    -1.730865         0.073375         -0.220875     -0.207142   
1    -1.728492        -0.872563          0.460320     -0.091886   
2    -1.726120         0.073375         -0.084636      0.073480   
3    -1.723747         0.309859         -0.447940     -0.096897   
4    -1.721374         0.073375          0.641972      0.375148   
...        ...              ...               ...           ...   
1455  1.721374         0.073375         -0.357114     -0.260560   
1456  1.723747        -0.872563          0.687385      0.266407   
1457  1.726120         0.309859         -0.175462     -0.147810   
1458  1.728492        -0.872563         -0.084636     -0.080160   
1459  1.730865        -0.872563          0.233255     -0.058112   

      num__OverallQual  num__OverallCond  num__YearBuilt  num__YearRemodAdd  \
0             0.651479         -0.517200        1.050994           0.878668   
1            -0.071836          2.179

**Задание**: Создайте новый трансформер, который для категориальных столбцов будет заполнять пропущенные значения наиболее часто встречаюмшщимся или новой категорией (сделайте параметром).

In [ ]:
# Заполнение пропущенных значений только наиболее часто встречающимися
df = pd.read_csv(cfg['house_pricing']['train_dataset'])

cat_features = df.select_dtypes(exclude=['number']).columns

cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='most_frequent'))])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
], remainder='passthrough')

preprocessor.set_output(transform='pandas')
df_transformed = preprocessor.fit_transform(df)

# print(df)
print(df_transformed)

     cat__MSZoning cat__Street cat__Alley cat__LotShape cat__LandContour  \
0               RL        Pave       Grvl           Reg              Lvl   
1               RL        Pave       Grvl           Reg              Lvl   
2               RL        Pave       Grvl           IR1              Lvl   
3               RL        Pave       Grvl           IR1              Lvl   
4               RL        Pave       Grvl           IR1              Lvl   
...            ...         ...        ...           ...              ...   
1455            RL        Pave       Grvl           Reg              Lvl   
1456            RL        Pave       Grvl           Reg              Lvl   
1457            RL        Pave       Grvl           Reg              Lvl   
1458            RL        Pave       Grvl           Reg              Lvl   
1459            RL        Pave       Grvl           Reg              Lvl   

     cat__Utilities cat__LotConfig cat__LandSlope cat__Neighborhood  \
0            All

In [213]:
class CategoricalImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy="most_frequent", fill_value="Unknown"):
        self.strategy = strategy
        self.fill_value = fill_value
        self.fill_dict_ = {}

    def fit(self, X, y=None):
        if self.strategy == "most_frequent":
            self.fill_dict_ = X.mode().iloc[0].to_dict()
        else:
            self.fill_dict_ = {col: self.fill_value for col in X.columns}
        
        return self

    def transform(self, X):
        return X.fillna(self.fill_dict_)

df = pd.read_csv(cfg['house_pricing']['train_dataset'])

cat_features = df.select_dtypes(include=['object']).columns

cat_pipeline = Pipeline([
    ('imputer', CategoricalImputer(strategy="most_frequent"))
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
], remainder='passthrough')

df_transformed = preprocessor.fit_transform(df)

print(df_transformed)

[['RL' 'Pave' 'Grvl' ... 2.0 2008.0 208500.0]
 ['RL' 'Pave' 'Grvl' ... 5.0 2007.0 181500.0]
 ['RL' 'Pave' 'Grvl' ... 9.0 2008.0 223500.0]
 ...
 ['RL' 'Pave' 'Grvl' ... 5.0 2010.0 266500.0]
 ['RL' 'Pave' 'Grvl' ... 4.0 2010.0 142125.0]
 ['RL' 'Pave' 'Grvl' ... 6.0 2008.0 147500.0]]


**Задание**: Создайте пайплайн по преобразованию категориальных столбцов, содержащий ваш импьютер и OneHotEncoder.

In [223]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])

cat_features = df.select_dtypes(include=['object']).columns

cat_pipeline = Pipeline([
    ('imputer', CategoricalImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
], remainder='passthrough')

df_transformed = preprocessor.fit_transform(df)

df_transformed = pd.DataFrame(df_transformed)

print(df_transformed.head())

   0    1    2    3    4    5    6    7    8    9    ...    279   280    281  \
0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  ...    0.0  61.0    0.0   
1  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  ...  298.0   0.0    0.0   
2  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  1.0  ...    0.0  42.0    0.0   
3  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  1.0  ...    0.0  35.0  272.0   
4  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  1.0  ...  192.0  84.0    0.0   

   282  283  284  285   286     287       288  
0  0.0  0.0  0.0  0.0   2.0  2008.0  208500.0  
1  0.0  0.0  0.0  0.0   5.0  2007.0  181500.0  
2  0.0  0.0  0.0  0.0   9.0  2008.0  223500.0  
3  0.0  0.0  0.0  0.0   2.0  2006.0  140000.0  
4  0.0  0.0  0.0  0.0  12.0  2008.0  250000.0  

[5 rows x 289 columns]


**Задание**: Создайте ColumnTransformer, который будет содержать в себе два вышеуказанных пайплайна.

In [226]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])

cat_features = df.select_dtypes(include=['object']).columns
num_features = df.select_dtypes(include=['number']).columns

cat_pipeline = Pipeline([
    ('imputer', CategoricalImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features),
    ('num', num_pipeline, num_features)
], remainder='passthrough')

df_transformed = preprocessor.fit_transform(df)

print(df_transformed)

[[ 0.          0.          0.         ... -1.5991111   0.13877749
   0.34727322]
 [ 0.          0.          0.         ... -0.48911005 -0.61443862
   0.00728832]
 [ 0.          0.          0.         ...  0.99089135  0.13877749
   0.53615372]
 ...
 [ 0.          0.          0.         ... -0.48911005  1.64520971
   1.07761115]
 [ 0.          0.          0.         ... -0.8591104   1.64520971
  -0.48852299]
 [ 0.          0.          0.         ... -0.1191097   0.13877749
  -0.42084081]]


**Доп.задание**: Используйте для комбинации результатов двух отдельных трансформеров FeatureUnion